## Explore Squad v1.1

In [112]:
import json

In [27]:
import os, sys
path_project = '~/fontes/exqa-complearning/'

sys.path.append(os.path.abspath('.'))
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath(r'../..'))
print(sys.path)

['/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/miniconda3/envs/relevar-busca/lib/python37.zip', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/lib-dynload', '', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages/IPython/extensions', '/home/borela/.ipython', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration/.\\.', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration/.\\.', '/home/borela/fontes/exqa-complearning/source/exploration'

In [23]:
%cd ..


/home/borela/fontes/exqa-complearning


In [28]:
%pwd

'/home/borela/fontes/exqa-complearning'

In [30]:
from transformers.data.processors.squad import SquadV1Processor # SquadV2Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples("data/dataset/squad/", filename="dev-v1.1.json")
print(len(examples))

100%|██████████| 48/48 [00:02<00:00, 23.14it/s]

10570


In [31]:
def display_example(qid):    
    from pprint import pprint

    idx = qid_to_example_index[qid]
    q = examples[idx].question_text
    c = examples[idx].context_text
    a = [answer['text'] for answer in examples[idx].answers]
    
    print(f'Example {idx} of {len(examples)}\n---------------------')
    print(f"Q: {q}\n")
    print("Context:")
    pprint(c)
    print(f"\nTrue Answers:\n{a}")



In [32]:
    
# generate some maps to help us identify examples of interest
qid_to_example_index = {example.qas_id: i for i, example in enumerate(examples)}
qid_to_has_answer = {example.qas_id: bool(example.answers) for example in examples}
answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if has_answer]
# no_answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if not has_answer]


In [33]:
display_example(answer_qids[1300])


Example 1300 of 10570
---------------------
Q: Where was the  Continental Edison Company located?

Context:
('In 1882, Tesla began working for the Continental Edison Company in France, '
 'designing and making improvements to electrical equipment. In June 1884, he '
 'relocated to New York City:57–60 where he was hired by Thomas Edison to work '
 "at his Edison Machine Works on Manhattan's lower east side. Tesla's work for "
 'Edison began with simple electrical engineering and quickly progressed to '
 'solving more difficult problems.')

True Answers:
['France', 'in France', 'France']


In [43]:
answer_qids[1300], type(answer_qids[1300])

('56e0d54a7aa994140058e76a', str)

In [44]:
qid = answer_qids[1300]

In [45]:
idx = qid_to_example_index[qid]
q = examples[idx].question_text
c = examples[idx].context_text
a = [answer['text'] for answer in examples[idx].answers]
q, c


('Where was the  Continental Edison Company located?',
 "In 1882, Tesla began working for the Continental Edison Company in France, designing and making improvements to electrical equipment. In June 1884, he relocated to New York City:57–60 where he was hired by Thomas Edison to work at his Edison Machine Works on Manhattan's lower east side. Tesla's work for Edison began with simple electrical engineering and quickly progressed to solving more difficult problems.")

In [36]:
import copy


In [37]:
from source.fine_tuning.responder_extracao import responder_extracao, json_exemplo  # pylint: disable=wrong-import-position # precisa dos sys.path antes
from source.fine_tuning.modelo_reader import lista_modelos_reader

modelo reader inicializados: pierreguillou/bert-large-cased-squad-v1.1-portuguese Hora: [2022-Jun-26 18:46:28]


In [38]:
json_fine_tuning = copy.deepcopy(json_exemplo)
json_fine_tuning["tamanho_max_resposta"] = 30
json_fine_tuning['texto_pergunta'] = q
json_fine_tuning['texto_contexto'] = c

In [49]:
json_fine_tuning

{'texto_pergunta': 'Where was the  Continental Edison Company located?',
 'top_k_reader': 3,
 'tamanho_max_resposta': 30,
 'texto_contexto': "In 1882, Tesla began working for the Continental Edison Company in France, designing and making improvements to electrical equipment. In June 1884, he relocated to New York City:57–60 where he was hired by Thomas Edison to work at his Edison Machine Works on Manhattan's lower east side. Tesla's work for Edison began with simple electrical engineering and quickly progressed to solving more difficult problems.",
 'sigla_modelo_reader': 'pierreguillou/bert-large-cased-squad-v1.1-portuguese'}

In [48]:
resposta = responder_extracao(json_fine_tuning)
print(f"resposta {resposta}")


resposta {'lista_sugestao_resposta': [{'texto_resposta': 'New York City:57–60', 'score': 0.9129816889762878, 'lista_referencia': [[164, 183]]}, {'texto_resposta': 'New York City:57–60 where he was hired by Thomas Edison to work at his Edison Machine Works', 'score': 0.0031480903271585703, 'lista_referencia': [[164, 255]]}]}


In [51]:
json_fine_tuning['texto_pergunta']  = "Em 1882, Tesla começou a trabalhar para a Continental Edison Company na França, projetando e fazendo melhorias em equipamentos elétricos. Em junho de 1884, ele se mudou para Nova York: 57-60, onde foi contratado por Thomas Edison para trabalhar em sua Edison Machine Works, no lado leste inferior de Manhattan. O trabalho de Tesla para Edison começou com engenharia elétrica simples e rapidamente progrediu para resolver problemas mais difíceis."
json_fine_tuning['texto_contexto'] = "Onde estava localizada a Continental Edison Company?"

In [50]:
a

['France', 'in France', 'France']

In [52]:
resposta = responder_extracao(json_fine_tuning)
print(f"resposta {resposta}")


resposta {'lista_sugestao_resposta': [{'texto_resposta': '', 'score': 0.22037574648857117, 'lista_referencia': [[0, 0]]}, {'texto_resposta': 'Onde estava localizada a Continental Edison Company?', 'score': 0.02530529536306858, 'lista_referencia': [[0, 51]]}, {'texto_resposta': 'localizada a Continental Edison Company?', 'score': 0.01168345008045435, 'lista_referencia': [[12, 51]]}]}


In [98]:
from source.calculation.metric import qa_metric
# exact_match_score, f1_score, metric_max_over_ground_truths


In [93]:
import importlib


In [99]:
importlib.reload(qa_metric)

nova importação
def


<module 'source.calculation.metric.qa_metric' from '/home/borela/fontes/exqa-complearning/source/calculation/metric/qa_metric.py'>

In [101]:
qa_metric.f1_score (prediction=resposta[1]['texto_resposta'],ground_truth=a[0])

0

In [104]:
qa_metric.exact_match_score(prediction=resposta[1]['texto_resposta'],ground_truth=a[0])

nova importação
def


0

In [55]:
resposta[1]['texto_resposta']

'Onde estava localizada a Continental Edison Company?'

In [109]:
qa_metric.metric_score_over_ground_truths(metric_fn= qa_metric.f1_score, prediction='france',ground_truths=a)

[1.0, 0.6666666666666666, 1.0]

In [106]:
qa_metric.metric_max_over_ground_truths(metric_fn= qa_metric.f1_score, prediction='france',ground_truths=a)

1.0

In [119]:
path_dataset_file = "data/dataset/squad/dev-v1.1.json"

expected_version= '1.1'

In [120]:
with open(path_dataset_file) as dataset_file:
    dataset_json = json.load(dataset_file)
    if (dataset_json['version'] != expected_version):
        print('Evaluation expects v-' + expected_version +
                ', but got dataset with v-' + dataset_json['version'],
                file=sys.stderr)
    dataset = dataset_json['data']


In [117]:
dataset_json.keys()


dict_keys(['data', 'version'])

In [121]:
len(dataset)


48

In [127]:
dataset[0].keys()

dict_keys(['title', 'paragraphs'])

In [126]:
dataset[0]['paragraphs'][1].keys()

dict_keys(['context', 'qas'])

In [142]:
print(f"Qtd artigos: {len(dataset)}")
for artigo in dataset:
    print(f"artigo.keys(): {artigo.keys()} Exemplo 1o artigo - Title: {artigo['title']}, #paragrafos {len(artigo['paragraphs'])} ")
    for paragrafo in artigo['paragraphs']:
        print(f"paragrafo.keys(): {paragrafo.keys()} Exemplo 1o paragrafo: qas: {len(paragrafo['qas'])} {paragrafo['context']}")
        for pergunta_respostas in paragrafo['qas']:
            print(f"pergunta_respostas.keys(): {pergunta_respostas.keys()} Exemplo 1a pergunta_respostas: qtd respostas: {len(pergunta_respostas['answers'])}; {pergunta_respostas['question']}")
            for resposta in pergunta_respostas['answers']:
                print(f"resposta.keys(): {resposta.keys()} Exemplo 1a resposta {resposta}")
                break
            ground_truths = list(map(lambda x: x['text'], pergunta_respostas['answers']))
            print(f"Exemplo 1o ground_truths {ground_truths}")
            break
        break
    break

Qtd artigos: 48
Exemplo 1o artigo - Title: Super_Bowl_50, #paragrafos 54 
Exemplo 1o paragrafo: qas: 30 Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
pergunta_respostas.keys(): dict_keys(['answers', 'question', 'id']) Exemplo 1a pergunta_respostas: qtd respostas: 3; Whi

In [ ]:
def evaluate(dataset, predictions):
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                # calcular resposta

                json_fine_tuning = copy.deepcopy(json_exemplo)
                json_fine_tuning["tamanho_max_resposta"] = 30
                json_fine_tuning['texto_pergunta'] = qa['question']
                json_fine_tuning['texto_contexto'] = c


                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

In [ ]:
with open(args.prediction_file) as prediction_file:
    predictions = json.load(prediction_file)
print(json.dumps(evaluate(dataset, predictions)))


## Config ...